In [3]:
import pandas as pd
import numpy as np
import requests
import time
import re
from datetime import datetime
from bs4 import BeautifulSoup
import os
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
import datetime
from datetime import date
from datetime import time
from datetime import timedelta
import random

##Models
import xgboost as xgb
from sklearn import linear_model
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

In [4]:
##Read in the player list 
pdf = pd.read_csv("NBA_Player_List_2015_2021.csv")
print(pdf.shape)
pdf =pdf[["bbref_code","Season","Pos","Wt","Ht"]]

(3963, 12)


In [5]:
print(pdf.shape)
pdf =pdf.drop_duplicates()
pdf=pdf.groupby(['bbref_code',"Season"]).nth(0).reset_index()
print(pdf.shape)

(3963, 5)
(3553, 5)


In [6]:
####
df = pd.read_csv("NBA_Player_Gamelog_2015_2021.csv")
df=df.groupby(['bbref_code',"Date"]).nth(0).reset_index()
print(df.shape)
df = df.merge(pdf,left_on=("bbref_code","season"),right_on=("bbref_code","Season"),how='left')
print(df.shape)
df["Date"]=pd.to_datetime(df["Date"],
                                format= '%Y-%m-%d',errors = 'ignore')
df=df.reset_index(drop=True)
df['row_num'] = np.arange(len(df))
df =df.set_index('row_num')
print(df.shape)
df =df.drop_duplicates()

print(df.shape)

df.head()

(229262, 49)
(229262, 53)
(229262, 53)
(229262, 53)


,bbref_code,Date,bbref_letter,season,G,Age,Age_int,Tm,Home/Away,Opp,Outcome,GS,DK points,Bonus counter,MP,seconds_played,FG,FGA,FG%,3P,3PA,3P%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,GmSc,+/-,TS%,eFG%,ORB%,DRB%,TRB%,AST%,STL%,BLK%,TOV%,USG%,ORtg,DRtg,BPM,Season,Pos,Wt,Ht
row_num,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,abrinal01,2016-10-26,a,2017.000,1.000,23-086,8481,OKC,Away,PHI,W (+6),0,3.750,0.000,13:24,804.000,1.000,2.000,0.500,1.000,1.000,1.000,0.000,0.000,nan,0.000,1.000,1.000,0.000,0.000,0.000,2.000,3.000,3.000,-0.900,3.000,0.750,0.750,0.000,8.000,3.700,0.000,0.000,0.000,50.000,12.400,64.000,101.000,-12.500,2017,SG,200.000,6-6
1,abrinal01,2016-10-28,a,2017.000,nan,23-088,8483,OKC,Home,PHO,W (+3),Did Not Play,nan,0.000,NaN,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,2017,SG,200.000,6-6
2,abrinal01,2016-10-30,a,2017.000,2.000,23-090,8485,OKC,Home,LAL,W (+17),0,7.000,0.000,1:49,109.000,1.000,1.000,1.000,0.000,0.000,nan,3.000,3.000,1.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,5.000,5.700,0.000,1.078,1.000,0.000,0.000,0.000,0.000,25.000,0.000,0.000,50.500,235.000,49.000,85.600,2017,SG,200.000,6-6
3,abrinal01,2016-11-02,a,2017.000,3.000,23-093,8488,OKC,Away,LAC,W (+2),0,6.000,0.000,7:54,474.000,2.000,2.000,1.000,2.000,2.000,1.000,0.000,0.000,nan,0.000,0.000,0.000,0.000,0.000,0.000,2.000,0.000,6.000,3.400,4.000,1.500,1.500,0.000,0.000,0.000,0.000,0.000,0.000,50.000,21.400,101.000,91.000,12.800,2017,SG,200.000,6-6
4,abrinal01,2016-11-03,a,2017.000,4.000,23-094,8489,OKC,Away,GSW,L (-26),0,10.750,0.000,19:20,1160.000,1.000,6.000,0.167,1.000,6.000,0.167,2.000,2.000,1.000,1.000,2.000,3.000,1.000,0.000,0.000,0.000,0.000,5.000,3.200,-13.000,0.363,0.250,5.600,11.500,8.600,7.600,0.000,0.000,0.000,14.800,100.000,132.000,-6.000,2017,SG,200.000,6-6


In [7]:
###Convert Ht to inches
HT = []
for v1 in df["Ht"]:
    #print(v1)
    HT.append((int(v1.split("-")[0])*12)+(int(v1.split("-")[1])))
df["Ht"] = HT

In [8]:
###Tranform some of the written data
df["Home/Away"] = np.where(df["Home/Away"]=='Away',1,0)

In [10]:
#Create an only numbers df
numdf = df[['bbref_code','season','Age_int','DK points', 'Bonus counter',
           'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF',
       'PTS', 'GmSc', '+/-', 'TS%', 'eFG%', 'ORB%', 'DRB%', 'TRB%', 'AST%',
       'STL%', 'BLK%', 'TOV%', 'USG%', 'ORtg', 'DRtg', 'BPM']]
#numdf=numdf.set_index(['bbref_code','season',"Date"])
numdf.head()

,bbref_code,season,Age_int,DK points,Bonus counter,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,GmSc,+/-,TS%,eFG%,ORB%,DRB%,TRB%,AST%,STL%,BLK%,TOV%,USG%,ORtg,DRtg,BPM
row_num,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,abrinal01,2017.000,8481,3.750,0.000,0.000,nan,0.000,1.000,1.000,0.000,0.000,0.000,2.000,3.000,3.000,-0.900,3.000,0.750,0.750,0.000,8.000,3.700,0.000,0.000,0.000,50.000,12.400,64.000,101.000,-12.500
1,abrinal01,2017.000,8483,nan,0.000,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
2,abrinal01,2017.000,8485,7.000,0.000,3.000,1.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,5.000,5.700,0.000,1.078,1.000,0.000,0.000,0.000,0.000,25.000,0.000,0.000,50.500,235.000,49.000,85.600
3,abrinal01,2017.000,8488,6.000,0.000,0.000,nan,0.000,0.000,0.000,0.000,0.000,0.000,2.000,0.000,6.000,3.400,4.000,1.500,1.500,0.000,0.000,0.000,0.000,0.000,0.000,50.000,21.400,101.000,91.000,12.800
4,abrinal01,2017.000,8489,10.750,0.000,2.000,1.000,1.000,2.000,3.000,1.000,0.000,0.000,0.000,0.000,5.000,3.200,-13.000,0.363,0.250,5.600,11.500,8.600,7.600,0.000,0.000,0.000,14.800,100.000,132.000,-6.000


In [11]:
############################################################################
############################################################################
############################################################################

In [16]:
dfroll2 = numdf.groupby(["bbref_code","season"], as_index=False).transform(lambda x: x.shift(1))
dfroll2["season"] = df["season"]
dfroll2["bbref_code"] = df["bbref_code"]
dfroll2 = dfroll2.groupby(["bbref_code","season"], as_index=False).transform(lambda x: x.rolling(2, min_periods=1).mean())
dfroll2 = dfroll2.add_prefix("roll2_")
dfroll2 = dfroll2.reset_index()
dfroll2 =dfroll2.set_index("row_num")

In [17]:
dfroll7 = numdf.groupby(["bbref_code","season"], as_index=False).transform(lambda x: x.shift(1))
dfroll7["season"] = df["season"]
dfroll7["bbref_code"] = df["bbref_code"]
dfroll7 = dfroll7.groupby(["bbref_code","season"], as_index=False).transform(lambda x: x.rolling(7, min_periods=1).mean())
dfroll7 = dfroll7.add_prefix("roll7_")
dfroll7 = dfroll7.reset_index()
dfroll7 =dfroll7.set_index("row_num")

In [18]:
dfseason = numdf.groupby(["bbref_code","season"], as_index=False).transform(lambda x: x.shift(1))
dfseason["season"] = df["season"]
dfseason["bbref_code"] = df["bbref_code"]
dfseason = dfseason.groupby(["bbref_code","season"], as_index=False).transform(lambda x: x.expanding(min_periods=1).mean())
dfseason = dfseason.add_prefix("season_")
dfseason = dfseason.reset_index()
dfseason =dfseason.set_index("row_num")

In [19]:
###Combine it all together
training = df[["DK points"]]
training=training.join(dfroll2,how='left')
training = training.join(dfroll7,how='left')
training = training.join(dfseason,how='left')
####
training =training[training['season_DK points']>=7]
#training =training[training['roll7_DK points']>=10]
###
training.head()

,DK points,roll2_Age_int,roll2_DK points,roll2_Bonus counter,roll2_FTA,roll2_FT%,roll2_ORB,roll2_DRB,roll2_TRB,roll2_AST,roll2_STL,roll2_BLK,roll2_TOV,roll2_PF,roll2_PTS,roll2_GmSc,roll2_+/-,roll2_TS%,roll2_eFG%,roll2_ORB%,roll2_DRB%,roll2_TRB%,roll2_AST%,roll2_STL%,roll2_BLK%,roll2_TOV%,roll2_USG%,roll2_ORtg,roll2_DRtg,roll2_BPM,roll7_Age_int,roll7_DK points,roll7_Bonus counter,roll7_FTA,roll7_FT%,roll7_ORB,roll7_DRB,roll7_TRB,roll7_AST,roll7_STL,roll7_BLK,roll7_TOV,roll7_PF,roll7_PTS,roll7_GmSc,roll7_+/-,roll7_TS%,roll7_eFG%,roll7_ORB%,roll7_DRB%,roll7_TRB%,roll7_AST%,roll7_STL%,roll7_BLK%,roll7_TOV%,roll7_USG%,roll7_ORtg,roll7_DRtg,roll7_BPM,season_Age_int,season_DK points,season_Bonus counter,season_FTA,season_FT%,season_ORB,season_DRB,season_TRB,season_AST,season_STL,season_BLK,season_TOV,season_PF,season_PTS,season_GmSc,season_+/-,season_TS%,season_eFG%,season_ORB%,season_DRB%,season_TRB%,season_AST%,season_STL%,season_BLK%,season_TOV%,season_USG%,season_ORtg,season_DRtg,season_BPM
row_num,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
6,9.000,8490.000,9.625,0.000,1.500,1.000,0.500,2.000,2.500,1.500,0.500,0.000,0.000,0.500,3.000,3.050,-8.500,0.355,0.125,2.800,11.650,7.700,10.000,1.500,0.000,0.000,9.250,116.000,115.500,-3.650,8486.167,7.200,0.000,1.200,1.000,0.200,1.000,1.200,0.600,0.400,0.000,0.800,0.800,4.000,2.860,-2.000,0.808,0.700,1.120,6.260,3.820,4.000,5.600,0.000,20.000,20.560,126.400,94.400,15.720,8486.167,7.200,0.000,1.200,1.000,0.200,1.000,1.200,0.600,0.400,0.000,0.800,0.800,4.000,2.860,-2.000,0.808,0.700,1.120,6.260,3.820,4.000,5.600,0.000,20.000,20.560,126.400,94.400,15.720
7,3.000,8492.000,8.750,0.000,0.500,1.000,0.500,1.500,2.000,1.000,1.000,0.000,1.000,1.500,3.000,0.800,-4.000,0.313,0.139,3.600,8.800,6.600,6.200,3.000,0.000,9.100,16.200,92.500,96.500,-8.250,8487.143,7.500,0.000,1.000,1.000,0.333,1.000,1.333,0.500,0.500,0.000,1.000,1.000,4.167,2.167,-2.333,0.719,0.630,2.133,6.183,4.250,3.333,5.167,0.000,19.700,21.917,114.167,94.333,10.567,8487.143,7.500,0.000,1.000,1.000,0.333,1.000,1.333,0.500,0.500,0.000,1.000,1.000,4.167,2.167,-2.333,0.719,0.630,2.133,6.183,4.250,3.333,5.167,0.000,19.700,21.917,114.167,94.333,10.567
11,11.500,8499.500,9.250,0.000,1.000,1.000,0.000,2.000,2.000,0.500,0.000,0.000,1.000,2.500,6.000,0.850,-5.500,0.409,0.357,0.000,13.300,6.150,4.600,0.000,0.000,11.900,19.850,78.500,117.500,-10.750,8494.857,7.643,0.000,1.000,0.875,0.286,1.429,1.714,0.571,0.286,0.000,0.857,1.429,4.143,1.186,-4.143,0.547,0.463,1.829,9.086,5.457,4.171,0.857,0.000,16.829,16.900,85.143,114.429,-6.843,8491.000,7.025,0.000,1.000,0.900,0.200,1.100,1.300,0.400,0.300,0.000,1.000,1.300,4.300,1.650,-2.200,0.715,0.649,1.280,7.160,4.190,2.920,3.100,0.000,21.780,20.260,99.600,104.200,3.800
12,13.000,8501.000,9.625,0.000,1.500,1.000,0.000,0.500,0.500,1.000,0.500,0.000,0.500,1.000,6.000,3.200,-4.500,0.553,0.482,0.000,3.300,1.400,9.550,1.650,0.000,5.650,17.800,110.500,107.000,-0.600,8496.714,7.750,0.000,0.857,0.875,0.143,1.143,1.286,0.571,0.429,0.000,0.857,1.429,4.429,1.686,-2.286,0.607,0.535,1.029,7.443,4.229,4.500,1.329,0.000,16.829,16.757,92.857,110.714,-4.657,8491.917,7.432,0.000,1.000,0.917,0.182,1.000,1.182,0.455,0.364,0.000,0.909,1.182,4.545,2.109,-2.000,0.722,0.658,1.164,6.509,3.809,3.555,3.118,0.000,19.800,19.673,104.545,104.364,4.300
13,11.000,8503.000,12.250,0.000,0.500,1.000,0.000,1.000,1.000,0.500,0.500,0.000,0.500,0.500,8.500,5.250,1.000,0.644,0.625,0.000,5.050,2.850,4.950,1.650,0.000,4.550,17.250,126.000,108.000,0.850,8498.571,8.393,0.000,0.714,0.833,0.143,1.143,1.286,0.286,0.286,0.000,1.000,1.429,5.714,1.814,-1.429,0.629,0.606,1.029,7.200,4.071,2.729,0.900,0.000,18.129,19.186,88.000,112.286,-5.557,8492.846,7.896,0.000,0.917,0.917,0.167,1.083,1.250,0.417,0.333,0.000,0.917,1.167,5.000,2.250,-1.667,0.704,0.645,1.067,6.808,3.967,3.258,2.858,0.000,18.908,19.758,104.000,104.833,3.308


In [20]:
###Add in the non numeric columns
training["Home/Away"] = df["Home/Away"]
training["Tm"] = df["Tm"]
training["Opp"] = df["Opp"]
training["Season"] = df["Season"]
training["Pos"] = df["Pos"]
training["Wt"] = df["Wt"]
training["Ht"] = df["Ht"]

In [21]:
###Handle Nulls
print(training.shape)
training =training[~training["DK points"].isnull()]
training =training.fillna(0)
print(training.shape)

(177861, 95)
(140793, 95)


In [22]:
training.groupby("Pos").count()

,DK points,roll2_Age_int,roll2_DK points,roll2_Bonus counter,roll2_FTA,roll2_FT%,roll2_ORB,roll2_DRB,roll2_TRB,roll2_AST,roll2_STL,roll2_BLK,roll2_TOV,roll2_PF,roll2_PTS,roll2_GmSc,roll2_+/-,roll2_TS%,roll2_eFG%,roll2_ORB%,roll2_DRB%,roll2_TRB%,roll2_AST%,roll2_STL%,roll2_BLK%,roll2_TOV%,roll2_USG%,roll2_ORtg,roll2_DRtg,roll2_BPM,roll7_Age_int,roll7_DK points,roll7_Bonus counter,roll7_FTA,roll7_FT%,roll7_ORB,roll7_DRB,roll7_TRB,roll7_AST,roll7_STL,roll7_BLK,roll7_TOV,roll7_PF,roll7_PTS,roll7_GmSc,roll7_+/-,roll7_TS%,roll7_eFG%,roll7_ORB%,roll7_DRB%,roll7_TRB%,roll7_AST%,roll7_STL%,roll7_BLK%,roll7_TOV%,roll7_USG%,roll7_ORtg,roll7_DRtg,roll7_BPM,season_Age_int,season_DK points,season_Bonus counter,season_FTA,season_FT%,season_ORB,season_DRB,season_TRB,season_AST,season_STL,season_BLK,season_TOV,season_PF,season_PTS,season_GmSc,season_+/-,season_TS%,season_eFG%,season_ORB%,season_DRB%,season_TRB%,season_AST%,season_STL%,season_BLK%,season_TOV%,season_USG%,season_ORtg,season_DRtg,season_BPM,Home/Away,Tm,Opp,Season,Wt,Ht
Pos,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
C,28513,28513,28513,28513,28513,28513,28513,28513,28513,28513,28513,28513,28513,28513,28513,28513,28513,28513,28513,28513,28513,28513,28513,28513,28513,28513,28513,28513,28513,28513,28513,28513,28513,28513,28513,28513,28513,28513,28513,28513,28513,28513,28513,28513,28513,28513,28513,28513,28513,28513,28513,28513,28513,28513,28513,28513,28513,28513,28513,28513,28513,28513,28513,28513,28513,28513,28513,28513,28513,28513,28513,28513,28513,28513,28513,28513,28513,28513,28513,28513,28513,28513,28513,28513,28513,28513,28513,28513,28513,28513,28513,28513,28513,28513
C-F,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13
F,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11
G,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5
G-F,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11
PF,28850,28850,28850,28850,28850,28850,28850,28850,28850,28850,28850,28850,28850,28850,28850,28850,28850,28850,28850,28850,28850,28850,28850,28850,28850,28850,28850,28850,28850,28850,28850,28850,28850,28850,28850,28850,28850,28850,28850,28850,28850,28850,28850,28850,28850,28850,28850,28850,28850,28850,28850,28850,28850,28850,28850,28850,28850,28850,28850,28850,28850,28850,28850,28850,28850,28850,28850,28850,28850,28850,28850,28850,28850,28850,28850,28850,28850,28850,28850,28850,28850,28850,28850,28850,28850,28850,28850,28850,28850,28850,28850,28850,28850,28850
PG,28351,28351,28351,28351,28351,28351,28351,28351,28351,28351,28351,28351,28351,28351,28351,28351,28351,28351,28351,28351,28351,28351,28351,28351,28351,28351,28351,28351,28351,28351,28351,28351,28351,28351,28351,28351,28351,28351,28351,28351,28351,28351,28351,28351,28351,28351,28351,28351,28351,28351,28351,28351,28351,28351,28351,28351,28351,28351,28351,28351,28351,28351,28351,28351,28351,28351,28351,28351,28351,28351,28351,28351,28351,28351,28351,28351,28351,28351,28351,28351,28351,28351,28351,28351,28351,28351,28351,28351,28351,28351,28351,28351,28351,28351
SF,23985,23985,23985,23985,23985,23985,23985,23985,23985,23985,23985,23985,23985,23985,23985,23985,23985,23985,23985,23985,

In [38]:
##################################################################
x_data = training[[ 'roll2_Age_int','roll7_Age_int' ,'roll2_DK points', 'roll2_Bonus counter',
       'roll2_FTA', 'roll2_FT%', 'roll2_ORB', 'roll2_DRB', 'roll2_TRB',
       'roll2_AST', 'roll2_STL', 'roll2_BLK', 'roll2_TOV', 'roll2_PF',
       'roll2_PTS', 'roll2_GmSc', 'roll2_+/-', 'roll2_TS%', 'roll2_eFG%',
       'roll2_ORB%', 'roll2_DRB%', 'roll2_TRB%', 'roll2_AST%', 'roll2_STL%',
       'roll2_BLK%', 'roll2_TOV%', 'roll2_USG%', 'roll2_ORtg', 'roll2_DRtg',
       'roll2_BPM',  'roll7_DK points', 'roll7_Bonus counter',
       'roll7_FTA', 'roll7_FT%', 'roll7_ORB', 'roll7_DRB', 'roll7_TRB',
       'roll7_AST', 'roll7_STL', 'roll7_BLK', 'roll7_TOV', 'roll7_PF',
       'roll7_PTS', 'roll7_GmSc', 'roll7_+/-', 'roll7_TS%', 'roll7_eFG%',
       'roll7_ORB%', 'roll7_DRB%', 'roll7_TRB%', 'roll7_AST%', 'roll7_STL%',
       'roll7_BLK%', 'roll7_TOV%', 'roll7_USG%', 'roll7_ORtg', 'roll7_DRtg',
       'roll7_BPM',
        'season_Age_int', 'season_DK points', 'season_Bonus counter',
       'season_FTA', 'season_FT%', 'season_ORB', 'season_DRB', 'season_TRB',
       'season_AST', 'season_STL', 'season_BLK', 'season_TOV', 'season_PF',
       'season_PTS', 'season_GmSc', 'season_+/-', 'season_TS%', 'season_eFG%',
       'season_ORB%', 'season_DRB%', 'season_TRB%', 'season_AST%',
       'season_STL%', 'season_BLK%', 'season_TOV%', 'season_USG%',
       'season_ORtg', 'season_DRtg', 'season_BPM'
                  ,"Season","Pos","Wt","Ht","Home/Away","Tm","Opp"]]
#x_data =x_data[~x_data["Pos"].isin(["SG","SF"])]
#,"Home/Away","Tm","Opp"
##Convert column to dummies
x_data = pd.get_dummies(x_data, columns=["Season"])
x_data = pd.get_dummies(x_data, columns=["Pos"])
x_data = pd.get_dummies(x_data, columns=["Home/Away"])
x_data = pd.get_dummies(x_data, columns=["Tm"])
x_data = pd.get_dummies(x_data, columns=["Opp"])

##Grab the perdiction column
y_data = training[["DK points"]]
#y_data = training[~training["Pos"].isin(["SG","SF"])][["DK points"]]

x_train, x_test, y_train, y_test = train_test_split(x_data,y_data,
                                test_size=0.33, random_state=42)


In [52]:
###########################################################################
###########################################################################
###########################################################################
################### Start the Models             ##########################

In [34]:
###########################################################################

In [41]:
regr = linear_model.LinearRegression()
regr.fit(x_train,y_train)
lintrainpreddf = regr.predict(x_train)
x=mean_squared_error(lintrainpreddf,y_train)
sqrt=x**(1/2)
print("Training linear")
print(sqrt)
lintrainpreddftest = regr.predict(x_test)
x_test["Linear_Pred"] = lintrainpreddftest
lintrainpreddftrain = regr.predict(x_train)
x_train["Linear_Pred"] = lintrainpreddftrain

Training linear
8.989512225980857


<ipython-input-41-fb5ece8e32dd>:9: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-41-fb5ece8e32dd>:11: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [42]:
x=mean_squared_error(y_test["DK points"],x_test["Linear_Pred"])
sqrt=x**(1/2)
print("train ravg")
print(sqrt)
x=mean_squared_error(y_train["DK points"],x_train["Linear_Pred"])
sqrt=x**(1/2)
print("train ravg")
print(sqrt)

train ravg
9.700168658673233
train ravg
8.989512225980857


In [37]:
###########################################################################

In [39]:
model3 = xgb.XGBRegressor(n_estimators=1000, learning_rate=0.01,gamma=0, subsample=0.75,
                           colsample_bytree=1, max_depth=5,nthread=8)
model3.fit(x_train,y_train)
trainpreddf3 = model3.predict(x_train)
x=mean_squared_error(trainpreddf3,y_train)
sqrt=x**(1/2)
print("Train XG")
print(sqrt)
testpreddf3 = model3.predict(x_test)
x_test["xgb_Pred"] = testpreddf3
x_train["xgb_Pred"] = model3.predict(x_train)

x=mean_squared_error(y_test["DK points"],x_test["xgb_Pred"])
sqrt=x**(1/2)
print("train ravg")
print(sqrt)
x=mean_squared_error(y_train["DK points"],x_train["xgb_Pred"])
sqrt=x**(1/2)
print("train ravg")
print(sqrt)


Train XG
9.234138342665666


<ipython-input-39-8fa12a70ec6d>:10: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



train ravg
9.4934287723241
train ravg
9.234138342665666


<ipython-input-39-8fa12a70ec6d>:11: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [40]:
##Call in the function
import plotly.express as px
###Run the ploty
feature_important = model3.get_booster().get_score(importance_type='weight')
keys = list(feature_important.keys())
values = list(feature_important.values())

data = pd.DataFrame(data=values, index=keys, columns=["score"]).sort_values(by = "score", ascending=True)
data= data.reset_index()
data.columns = ["Feature","score"]
data = data.sort_values("score",ascending=0)
fig = px.bar(data, x="Feature", y="score", title='Test')
fig.show()

In [40]:
###########################################################################

In [29]:
model = RandomForestRegressor(n_estimators=100,
                             n_jobs=-1,
                             random_state=42,
                             max_features="auto",                           
                             )
model.fit(x_train,y_train)
trainpreddf = model.predict(x_train)
x=mean_squared_error(trainpreddf,y_train)
sqrt=x**(1/2)
print("Training RF")
print(sqrt)
testpreddf = model.predict(x_test)
x=mean_squared_error(testpreddf,y_test)
sqrt=x**(1/2)
print("test RF")
print(sqrt)

<ipython-input-29-757c86ea86df>:6: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



Training RF
3.32532404295749
test RF
9.859020859679392


In [30]:
###Send the predictions
x_test["random_Pred"] =model.predict(x_test)
x_train["random_Pred"] = model.predict(x_train)


<ipython-input-30-b83d437532b6>:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-30-b83d437532b6>:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [31]:
####Robust Linear#########################################################

In [43]:
ransac = linear_model.RANSACRegressor()
ransac.fit(x_train,y_train)
inlier_mask = ransac.inlier_mask_
outlier_mask = np.logical_not(inlier_mask)

###Send the predictions
x_test["robust_Pred"] = ransac.predict(x_test)
x_train["robust_Pred"] = ransac.predict(x_train)


####Check the mean squared error
x=mean_squared_error(y_test["DK points"],x_test["robust_Pred"])
sqrt=x**(1/2)
print("test ravg")
print(sqrt)
x=mean_squared_error(y_train["DK points"],x_train["robust_Pred"])
sqrt=x**(1/2)
print("train ravg")
print(sqrt)


test ravg
11051265.298966037
train ravg
7438096.361281158


<ipython-input-43-b2be2d23739b>:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-43-b2be2d23739b>:8: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



###Send the predictions
x_test["robust_Pred"] = abs(x_test["robust_Pred"])
x_train["robust_Pred"] = abs(x_train["robust_Pred"])

x_test = x_test.drop("robust_Pred",axis=1) 
x_train = x_train.drop("robust_Pred",axis=1) 

In [33]:
###########################################################################

In [44]:
clf = linear_model.Lasso(alpha=0.01,normalize=True)
clf.fit(x_train,y_train)
#lassopred = clf.predict(x_train)



###Send the predictions
x_test["lasso_Pred"] = clf.predict(x_test)
x_train["lasso_Pred"] = clf.predict(x_train)


####Check the mean squared error
x=mean_squared_error(y_test["DK points"],x_test["lasso_Pred"])
sqrt=x**(1/2)
print("test ravg")
print(sqrt)
x=mean_squared_error(y_train["DK points"],x_train["lasso_Pred"])
sqrt=x**(1/2)
print("train ravg")
print(sqrt)


test ravg
9.98893040500171
train ravg
9.499706840795623


<ipython-input-44-762141d595ce>:8: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-44-762141d595ce>:9: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [35]:
###########################################################################

In [36]:
#Do another xgboost because it just works

In [45]:
modelE = xgb.XGBRegressor(n_estimators=750, learning_rate=0.01,gamma=0, 
                          subsample=0.75,
                           colsample_bytree=1, max_depth=5,nthread=8)
modelE.fit(x_train,y_train)
trainpreddf3 = modelE.predict(x_train)
x=mean_squared_error(trainpreddf3,y_train)
sqrt=x**(1/2)
print("Train XG")
print(sqrt)
testpreddf3 = modelE.predict(x_test)
x_test["xgb_Pred"] = testpreddf3
x_train["xgb_Pred"] = modelE.predict(x_train)

x=mean_squared_error(y_test["DK points"],x_test["xgb_Pred"])
sqrt=x**(1/2)
print("test ravg")
print(sqrt)
x=mean_squared_error(y_train["DK points"],x_train["xgb_Pred"])
sqrt=x**(1/2)
print("train ravg")
print(sqrt)


Train XG
8.639788041924245


<ipython-input-45-c6e24d91432c>:11: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



test ravg
9.692353035927594
train ravg
8.639788041924245


<ipython-input-45-c6e24d91432c>:12: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
10.1

In [50]:
###########################################################################

In [116]:
gapchecker["Pos"]= df["Pos"]

In [123]:
gapchecker =x_test.copy()

gapchecker["DK points"] =y_test["DK points"]
gapchecker["Dk_bucket"] = pd.qcut(gapchecker["DK points"],5)

gapchecker.groupby(["Dk_bucket"]).agg({"roll2_Age_int":'count'})
gapchecker["1"] = 1

In [124]:
####This allows you to calc the RMSE 
import numpy as np
import pandas as pd
from sklearn.metrics import r2_score, mean_squared_error

def r2_rmse( gapchecker ):
    r2 = r2_score( gapchecker['DK points'], gapchecker['xgb_Pred'] )
    rmse = np.sqrt( mean_squared_error( gapchecker['DK points'], gapchecker['xgb_Pred'] ) )
    return pd.Series( dict(  r2 = r2, rmse = rmse ) )

gapchecker.groupby(["Dk_bucket"]).apply( r2_rmse ).reset_index()

,Dk_bucket,r2,rmse
0,"(-1.501, 8.0]",-12.264,9.543
1,"(8.0, 15.25]",-12.476,7.694
2,"(15.25, 22.75]",-9.954,7.184
3,"(22.75, 32.25]",-9.445,8.854
4,"(32.25, 102.25]",-1.251,14.059


In [72]:
gapchecker.groupby("1").apply( r2_rmse ).reset_index()

,1,r2,rmse
0,1,0.517,9.924


In [86]:
gapchecker.groupby("1").apply( r2_rmse ).reset_index()

,1,r2,rmse
0,1,0.497,10.126


In [125]:
gapchecker.groupby("1").apply( r2_rmse ).reset_index()

,1,r2,rmse
0,1,0.531,9.772


In [126]:
def r2_rmse( gapchecker ):
    r2 = r2_score( gapchecker['DK points'], gapchecker["Man_pred"] )
    rmse = np.sqrt( mean_squared_error( gapchecker['DK points'],
                                       gapchecker["Man_pred"] ) )
    return pd.Series( dict(  r2 = r2, rmse = rmse ) )

gapchecker =x_test.copy()

gapchecker["DK points"] =y_test["DK points"]
gapchecker["Dk_bucket"] = pd.qcut(gapchecker["DK points"],5)

###
####Build a function to optimze results based on the manual pred method
manualpred = []
for v1, v2, v3,v4 in zip(gapchecker["DK points"],
        gapchecker["lasso_Pred"],gapchecker["xgb_Pred"],gapchecker["season_DK points"]):
    if v1 <= 7.5:
        manualpred.append(v3)
    elif v1 >= 30.75:
        manualpred.append(v3)
    elif  15 < v1 < 22.25:
        manualpred.append(v4)
    else:
        manualpred.append(v2)
gapchecker["Man_pred"] =manualpred

###
gapchecker["1"] = 1
gapchecker.groupby("Dk_bucket").apply( r2_rmse ).reset_index()

,Dk_bucket,r2,rmse
0,"(-1.501, 8.0]",-12.346,9.573
1,"(8.0, 15.25]",-11.114,7.295
2,"(15.25, 22.75]",-1.375,3.345
3,"(22.75, 32.25]",-6.600,7.552
4,"(32.25, 102.25]",-1.251,14.059


In [74]:
gapchecker.groupby("1").apply( r2_rmse ).reset_index()

,1,r2,rmse
0,1,0.590,9.146


In [88]:
gapchecker.groupby("1").apply( r2_rmse ).reset_index()

,1,r2,rmse
0,1,0.578,9.278


In [127]:
gapchecker.groupby("1").apply( r2_rmse ).reset_index()

,1,r2,rmse
0,1,0.598,9.057


In [75]:
####Build a function to optimze results based on the manual pred method
manualpred = []
for v1, v2, v3 in zip(gapchecker["DK points"],
                      gapchecker["lasso_Pred"],gapchecker["xgb_Pred"]):
    if v1 <= 7.5:
        manualpred.append(v3)
    elif v1 >= 30.75:
        manualpred.append(v3)
    else:
        manualpred.append(v2)
gapchecker["Man_pred"] =manualpred

In [ ]:
gapchecker

In [151]:
####Build a function to optimze results based on the manual pred method
manualpred = []
for v1, v2, v3,v4 in zip(gapchecker["DK points"],
        gapchecker["lasso_Pred"],gapchecker["xgb_Pred"],gapchecker["season_DK points"]):
    if v1 <= 7.5:
        manualpred.append(v3)
    elif v1 >= 30.75:
        manualpred.append(v3)
    elif  15 < v1 < 22.25:
        manualpred.append(v4)
    else:
        manualpred.append(v2)
gapchecker["Man_pred"] =manualpred

In [ ]:
######MAke predictions on real data now

In [132]:
gapchecker.head()

,roll2_Age_int,roll7_Age_int,roll2_DK points,roll2_Bonus counter,roll2_FTA,roll2_FT%,roll2_ORB,roll2_DRB,roll2_TRB,roll2_AST,roll2_STL,roll2_BLK,roll2_TOV,roll2_PF,roll2_PTS,roll2_GmSc,roll2_+/-,roll2_TS%,roll2_eFG%,roll2_ORB%,roll2_DRB%,roll2_TRB%,roll2_AST%,roll2_STL%,roll2_BLK%,roll2_TOV%,roll2_USG%,roll2_ORtg,roll2_DRtg,roll2_BPM,roll7_DK points,roll7_Bonus counter,roll7_FTA,roll7_FT%,roll7_ORB,roll7_DRB,roll7_TRB,roll7_AST,roll7_STL,roll7_BLK,roll7_TOV,roll7_PF,roll7_PTS,roll7_GmSc,roll7_+/-,roll7_TS%,roll7_eFG%,roll7_ORB%,roll7_DRB%,roll7_TRB%,roll7_AST%,roll7_STL%,roll7_BLK%,roll7_TOV%,roll7_USG%,roll7_ORtg,roll7_DRtg,roll7_BPM,season_Age_int,season_DK points,season_Bonus counter,season_FTA,season_FT%,season_ORB,season_DRB,season_TRB,season_AST,season_STL,season_BLK,season_TOV,season_PF,season_PTS,season_GmSc,season_+/-,season_TS%,season_eFG%,season_ORB%,season_DRB%,season_TRB%,season_AST%,season_STL%,season_BLK%,season_TOV%,season_USG%,season_ORtg,season_DRtg,season_BPM,Wt,Ht,Season_2015,Season_2016,Season_2017,Season_2018,Season_2019,Season_2020,Season_2021,Pos_C,Pos_C-F,Pos_F,Pos_G,Pos_G-F,Pos_PF,Pos_PG,Pos_SF,Pos_SG,Home/Away_0,Home/Away_1,Tm_ATL,Tm_BOS,Tm_BRK,Tm_CHI,Tm_CHO,Tm_CLE,Tm_DAL,Tm_DEN,Tm_DET,Tm_GSW,Tm_HOU,Tm_IND,Tm_LAC,Tm_LAL,Tm_MEM,Tm_MIA,Tm_MIL,Tm_MIN,Tm_NOP,Tm_NYK,Tm_OKC,Tm_ORL,Tm_PHI,Tm_PHO,Tm_POR,Tm_SAC,Tm_SAS,Tm_TOR,Tm_UTA,Tm_WAS,Opp_ATL,Opp_BOS,Opp_BRK,Opp_CHI,Opp_CHO,Opp_CLE,Opp_DAL,Opp_DEN,Opp_DET,Opp_GSW,Opp_HOU,Opp_IND,Opp_LAC,Opp_LAL,Opp_MEM,Opp_MIA,Opp_MIL,Opp_MIN,Opp_NOP,Opp_NYK,Opp_OKC,Opp_ORL,Opp_PHI,Opp_PHO,Opp_POR,Opp_SAC,Opp_SAS,Opp_TOR,Opp_UTA,Opp_WAS,xgb_Pred,Linear_Pred,robust_Pred,lasso_Pred,DK points,Dk_bucket,1,Pos
row_num,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
301991,8799.000,8793.714,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,35.650,0.429,4.600,0.760,0.800,7.800,8.600,1.000,1.000,1.000,1.000,4.200,18.400,12.400,2.000,0.543,0.502,2.140,22.600,12.400,3.800,1.240,2.380,5.180,21.520,106.800,109.400,-1.440,9833.556,19.780,0.079,2.032,0.753,0.922,3.026,3.948,2.075,0.703,0.425,1.239,1.864,9.524,7.294,-0.343,0.525,0.494,4.659,14.879,9.788,13.357,1.528,1.598,12.888,18.961,102.140,109.581,-1.607,203.000,79,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,14.825,17.073,30.466,18.089,28.000,"(21.5, 30.75]",1,SF
207948,9977.000,9972.143,14.250,0.000,1.000,1.000,0.000,1.000,1.000,1.500,1.000,0.000,0.000,2.000,8.500,6.200,-5.000,0.734,0.711,0.000,5.150,2.000,39.200,1.750,0.000,0.000,19.450,159.000,116.500,8.000,15.417,0.000,2.500,0.917,0.500,1.167,1.667,2.167,0.667,0.167,0.333,2.000,8.167,6.467,3.333,0.586,0.531,3.233,7.817,5.100,25.717,1.550,1.083,5.617,19.667,133.833,112.500,3.967,9883.862,19.388,0.077,2.024,0.749,0.922,2.963,3.885,2.045,0.701,0.419,1.244,1.836,9.318,7.105,-0.292,0.521,0.489,4.708,14.772,9.762,13.391,1.552,1.554,13.111,19.046,101.099,108.682,-1.576,175.000,72,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,14.245,13.077,17.188,15.545,16.500,"(14.5, 21.5]",1,PG
139984,7359.500,7354.286,27.500,0.000,5.500,0.928,0.000,4.000,4.000,4.500,1.500,0.000,2.000,3.500,13.500,6.150,-1.500,0.351,0.248,0.000,13.100,6.650,20.900,2.350,0.000,10.600,26.850,81.500,109.000,-7.700,34.393,0.000,5.000,0.907,0.714,3.143,3.857,4.857,1.000,0.143,2.571,3.429,20.429,12.057,-4.143,0.479,0.413,2.443,10.671,6.700,24.057,1.471,0.357,10.900,29.800,102.857,118.714,-2.314,9953.531,19.182,0.078,2.036,0.746,0.930,2.922,3.852,2.046,0.700,0.419,1.247,1.839,9.178,6.967,-0.299,0.516,0.483,4.739,14.564,9.676,13.497,1.567,1.524,13.212,19.048,100.255,1

In [141]:
training.head()

,DK points,roll2_Age_int,roll2_DK points,roll2_Bonus counter,roll2_FTA,roll2_FT%,roll2_ORB,roll2_DRB,roll2_TRB,roll2_AST,roll2_STL,roll2_BLK,roll2_TOV,roll2_PF,roll2_PTS,roll2_GmSc,roll2_+/-,roll2_TS%,roll2_eFG%,roll2_ORB%,roll2_DRB%,roll2_TRB%,roll2_AST%,roll2_STL%,roll2_BLK%,roll2_TOV%,roll2_USG%,roll2_ORtg,roll2_DRtg,roll2_BPM,roll7_Age_int,roll7_DK points,roll7_Bonus counter,roll7_FTA,roll7_FT%,roll7_ORB,roll7_DRB,roll7_TRB,roll7_AST,roll7_STL,roll7_BLK,roll7_TOV,roll7_PF,roll7_PTS,roll7_GmSc,roll7_+/-,roll7_TS%,roll7_eFG%,roll7_ORB%,roll7_DRB%,roll7_TRB%,roll7_AST%,roll7_STL%,roll7_BLK%,roll7_TOV%,roll7_USG%,roll7_ORtg,roll7_DRtg,roll7_BPM,season_Age_int,season_DK points,season_Bonus counter,season_FTA,season_FT%,season_ORB,season_DRB,season_TRB,season_AST,season_STL,season_BLK,season_TOV,season_PF,season_PTS,season_GmSc,season_+/-,season_TS%,season_eFG%,season_ORB%,season_DRB%,season_TRB%,season_AST%,season_STL%,season_BLK%,season_TOV%,season_USG%,season_ORtg,season_DRtg,season_BPM,Home/Away,Tm,Opp,Season,Pos,Wt,Ht,Date
row_num,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2,11.250,11773.500,11.250,0.000,3.000,0.667,0.000,1.000,1.000,2.000,0.000,0.000,1.000,3.000,7.000,4.100,-2.000,0.658,0.625,0.000,5.300,2.900,15.800,0.000,0.000,15.800,15.800,114.000,109.000,-2.800,11773.500,11.250,0.000,3.000,0.667,0.000,1.000,1.000,2.000,0.000,0.000,1.000,3.000,7.000,4.100,-2.000,0.658,0.625,0.000,5.300,2.900,15.800,0.000,0.000,15.800,15.800,114.000,109.000,-2.800,11773.500,11.250,0.000,3.000,0.667,0.000,1.000,1.000,2.000,0.000,0.000,1.000,3.000,7.000,4.100,-2.000,0.658,0.625,0.000,5.300,2.900,15.800,0.000,0.000,15.800,15.800,114.000,109.000,-2.800,1,ATL,SAS,2015,C,260.000,83,2014-11-05
3,18.000,11777.000,11.250,0.000,1.500,0.667,0.000,2.000,2.000,1.000,0.000,0.000,0.500,2.500,7.000,3.750,0.500,0.579,0.562,0.000,14.750,6.750,7.900,0.000,0.000,7.900,17.650,105.000,106.500,-2.700,11775.333,11.250,0.000,1.500,0.667,0.000,2.000,2.000,1.000,0.000,0.000,0.500,2.500,7.000,3.750,0.500,0.579,0.562,0.000,14.750,6.750,7.900,0.000,0.000,7.900,17.650,105.000,106.500,-2.700,11775.333,11.250,0.000,1.500,0.667,0.000,2.000,2.000,1.000,0.000,0.000,0.500,2.500,7.000,3.750,0.500,0.579,0.562,0.000,14.750,6.750,7.900,0.000,0.000,7.900,17.650,105.000,106.500,-2.700,1,ATL,CHO,2015,C,260.000,83,2014-11-07
4,5.500,11780.000,14.625,0.000,0.000,0.000,0.000,2.500,2.500,1.500,0.500,0.000,0.000,2.500,7.500,5.100,9.500,0.536,0.536,0.000,18.100,8.150,10.700,1.100,0.000,0.000,16.900,110.000,105.500,1.750,11776.750,13.500,0.000,1.000,0.667,0.000,2.000,2.000,1.667,0.333,0.000,0.333,2.667,7.333,4.767,5.667,0.576,0.565,0.000,13.833,6.400,12.400,0.733,0.000,5.267,16.533,111.333,106.667,0.233,11776.750,13.500,0.000,1.000,0.667,0.000,2.000,2.000,1.667,0.333,0.000,0.333,2.667,7.333,4.767,5.667,0.576,0.565,0.000,13.833,6.400,12.400,0.733,0.000,5.267,16.533,111.333,106.667,0.233,0,ATL,NYK,2015,C,260.000,83,2014-11-08
5,8.000,11781.500,11.750,0.000,1.500,0.667,0.500,1.500,2.000,1.500,0.500,0.500,1.000,2.500,5.000,2.600,18.500,0.401,0.286,3.150,9.250,6.050,10.700,1.100,2.150,15.800,15.350,85.000,110.000,-5.650,11777.800,11.500,0.000,1.500,0.667,0.250,1.750,2.000,1.250,0.250,0.250,0.750,2.500,6.000,3.175,9.500,0.490,0.424,1.575,12.000,6.400,9.300,0.550,1.075,11.850,16.500,95.000,108.250,-4.175,11777.800,11.500,0.000,1.500,0.667,0.250,1.750,2.000,1.250,0.250,0.250,0.750,2.500,6.000,3.175,9.500,0.490,0.424,1.575,12.000,6.400,9.300,0.550,1.075,11.850,16.500,95.000,108.250,-4.175,1,ATL,NYK,2015,C,260.000,83,2014-11-10
6,11.750,11783.000,6.750,0.000,1.500,0.667,0.500,1.500,2.000,0.000,0.000,0.500,1.000,2.000,3.500,0.600,15.500,0.428,0.313,3.150,10.500,6.950,0.000,0.000,2.150,15.800,14.100,82.000,107.000,-9.400,11778.833,10.800,0.000,1.200,0.667,0.200,1.800,2.000,1.000,0.200,0.200,0.600,2.400,5.800,3.100,9.600,0.517,0.464,1.260,12.500,6.620,7.440,0.440,0.860,9.480,15.560,99.600,106.800,-3.620,11778.833,10.800,0.000,1.200,0.667,0.200,1.80

In [146]:
training["Date"] = df["Date"]
unqdates=training[training["Season"]==2021]["Date"].unique()

In [147]:
training[training["Season"]==2021].groupby("Date").count().head()

,DK points,roll2_Age_int,roll2_DK points,roll2_Bonus counter,roll2_FTA,roll2_FT%,roll2_ORB,roll2_DRB,roll2_TRB,roll2_AST,roll2_STL,roll2_BLK,roll2_TOV,roll2_PF,roll2_PTS,roll2_GmSc,roll2_+/-,roll2_TS%,roll2_eFG%,roll2_ORB%,roll2_DRB%,roll2_TRB%,roll2_AST%,roll2_STL%,roll2_BLK%,roll2_TOV%,roll2_USG%,roll2_ORtg,roll2_DRtg,roll2_BPM,roll7_Age_int,roll7_DK points,roll7_Bonus counter,roll7_FTA,roll7_FT%,roll7_ORB,roll7_DRB,roll7_TRB,roll7_AST,roll7_STL,roll7_BLK,roll7_TOV,roll7_PF,roll7_PTS,roll7_GmSc,roll7_+/-,roll7_TS%,roll7_eFG%,roll7_ORB%,roll7_DRB%,roll7_TRB%,roll7_AST%,roll7_STL%,roll7_BLK%,roll7_TOV%,roll7_USG%,roll7_ORtg,roll7_DRtg,roll7_BPM,season_Age_int,season_DK points,season_Bonus counter,season_FTA,season_FT%,season_ORB,season_DRB,season_TRB,season_AST,season_STL,season_BLK,season_TOV,season_PF,season_PTS,season_GmSc,season_+/-,season_TS%,season_eFG%,season_ORB%,season_DRB%,season_TRB%,season_AST%,season_STL%,season_BLK%,season_TOV%,season_USG%,season_ORtg,season_DRtg,season_BPM,Home/Away,Tm,Opp,Season,Pos,Wt,Ht
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2020-12-22,53,53,53,53,53,53,53,53,53,53,53,53,53,53,53,53,53,53,53,53,53,53,53,53,53,53,53,53,53,53,53,53,53,53,53,53,53,53,53,53,53,53,53,53,53,53,53,53,53,53,53,53,53,53,53,53,53,53,53,53,53,53,53,53,53,53,53,53,53,53,53,53,53,53,53,53,53,53,53,53,53,53,53,53,53,53,53,53,53,53,53,53,53,53,53
2020-12-23,254,254,254,254,254,254,254,254,254,254,254,254,254,254,254,254,254,254,254,254,254,254,254,254,254,254,254,254,254,254,254,254,254,254,254,254,254,254,254,254,254,254,254,254,254,254,254,254,254,254,254,254,254,254,254,254,254,254,254,254,254,254,254,254,254,254,254,254,254,254,254,254,254,254,254,254,254,254,254,254,254,254,254,254,254,254,254,254,254,254,254,254,254,254,254
2020-12-25,130,130,130,130,130,130,130,130,130,130,130,130,130,130,130,130,130,130,130,130,130,130,130,130,130,130,130,130,130,130,130,130,130,130,130,130,130,130,130,130,130,130,130,130,130,130,130,130,130,130,130,130,130,130,130,130,130,130,130,130,130,130,130,130,130,130,130,130,130,130,130,130,130,130,130,130,130,130,130,130,130,130,130,130,130,130,130,130,130,130,130,130,130,130,130
2020-12-26,209,209,209,209,209,209,209,209,209,209,209,209,209,209,209,209,209,209,209,209,209,209,209,209,209,209,209,209,209,209,209,209,209,209,209,209,209,209,209,209,209,209,209,209,209,209,209,209,209,209,209,209,209,209,209,209,209,209,209,209,209,209,209,209,209,209,209,209,209,209,209,209,209,209,209,209,209,209,209,209,209,209,209,209,209,209,209,209,209,209,209,209,209,209,209
2020-12-27,229,229,229,229,229,229,229,229,229,229,229,229,229,229,229,229,229,229,229,229,229,229,229,229,229,229,229,229,229,229,229,229,229,229,229,229,229,229,229,229,229,229,229,229,229,229,229,229,229,229,229,229,229,229,229,229,229,229,229,229,229,229,229,229,229,229,229,229,229,229,229,229,229,229,229,229,229,229,229,229,229,229,229,229,229,229,229,229,229,229,229,229,229,229,229


In [151]:
pred = training[training["Date"]=="2020-12-27"]
pred= pred.drop("Date",axis=1)  

In [275]:
##################################################################
x_data = training[[ 'roll2_Age_int','roll7_Age_int' ,'roll2_DK points', 'roll2_Bonus counter',
       'roll2_FTA', 'roll2_FT%', 'roll2_ORB', 'roll2_DRB', 'roll2_TRB',
       'roll2_AST', 'roll2_STL', 'roll2_BLK', 'roll2_TOV', 'roll2_PF',
       'roll2_PTS', 'roll2_GmSc', 'roll2_+/-', 'roll2_TS%', 'roll2_eFG%',
       'roll2_ORB%', 'roll2_DRB%', 'roll2_TRB%', 'roll2_AST%', 'roll2_STL%',
       'roll2_BLK%', 'roll2_TOV%', 'roll2_USG%', 'roll2_ORtg', 'roll2_DRtg',
       'roll2_BPM',  'roll7_DK points', 'roll7_Bonus counter',
       'roll7_FTA', 'roll7_FT%', 'roll7_ORB', 'roll7_DRB', 'roll7_TRB',
       'roll7_AST', 'roll7_STL', 'roll7_BLK', 'roll7_TOV', 'roll7_PF',
       'roll7_PTS', 'roll7_GmSc', 'roll7_+/-', 'roll7_TS%', 'roll7_eFG%',
       'roll7_ORB%', 'roll7_DRB%', 'roll7_TRB%', 'roll7_AST%', 'roll7_STL%',
       'roll7_BLK%', 'roll7_TOV%', 'roll7_USG%', 'roll7_ORtg', 'roll7_DRtg',
       'roll7_BPM',
        'season_Age_int', 'season_DK points', 'season_Bonus counter',
       'season_FTA', 'season_FT%', 'season_ORB', 'season_DRB', 'season_TRB',
       'season_AST', 'season_STL', 'season_BLK', 'season_TOV', 'season_PF',
       'season_PTS', 'season_GmSc', 'season_+/-', 'season_TS%', 'season_eFG%',
       'season_ORB%', 'season_DRB%', 'season_TRB%', 'season_AST%',
       'season_STL%', 'season_BLK%', 'season_TOV%', 'season_USG%',
       'season_ORtg', 'season_DRtg', 'season_BPM'
                  ,"Season","Pos","Wt","Ht","Home/Away","Tm","Opp"]]
#x_data =x_data[~x_data["Pos"].isin(["SG","SF"])]
#,"Home/Away","Tm","Opp"
##Convert column to dummies
x_data = pd.get_dummies(x_data, columns=["Season"])
x_data = pd.get_dummies(x_data, columns=["Pos"])
x_data = pd.get_dummies(x_data, columns=["Home/Away"])
x_data = pd.get_dummies(x_data, columns=["Tm"])
x_data = pd.get_dummies(x_data, columns=["Opp"])

##Grab the perdiction column
y_data = training[["DK points"]]
#y_data = pred[~pred["Pos"].isin(["SG","SF"])][["DK points"]]

x_train, x_test, y_train, y_test = train_test_split(x_data,y_data,
                                test_size=0.33, random_state=42)


In [276]:
x_data["Date"] = df["Date"]
y_data["Date"] = df["Date"]

<ipython-input-276-929f8f97f9d7>:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [277]:
x_data =x_data[x_data["Date"]=="2020-12-27"]
x_data = x_data.drop("Date",axis=1)

y_data =y_data[y_data["Date"]=="2020-12-27"]
y_data = y_data.drop("Date",axis=1)

In [278]:
#1st xgx_data
x_data["xgb_Pred"] = model3.predict(x_data)
#Linear x_data
x_data["Linear_Pred"] = regr.predict(x_data)
#robust x_data
x_data["robust_Pred"] = ransac.predict(x_data)
#Lasso x_data
x_data["lasso_Pred"] = clf.predict(x_data)
#2nd Xg Final Ensamble
x_data["xgb_Pred"] = modelE.predict(x_data)

In [46]:

###Pickle the mode
import pickle

#####Save all the model
Pkl_Filename = "Pickle_NBA_XG1_Model2021.pkl"  

with open(Pkl_Filename, 'wb') as file:  
    pickle.dump(model3, file)

Pkl_Filename = "Pickle_NBA_Linear_Model2021.pkl"  

with open(Pkl_Filename, 'wb') as file:  
    pickle.dump(regr, file)

Pkl_Filename = "Pickle_NBA_robust_Model2021.pkl"  
with open(Pkl_Filename, 'wb') as file:  
    pickle.dump(ransac, file)
    
    
Pkl_Filename = "Pickle_NBA_lasso_Model2021.pkl" 
with open(Pkl_Filename, 'wb') as file:  
    pickle.dump(clf, file)
    
Pkl_Filename = "Pickle_NBA_XG2_Model2021.pkl"  
with open(Pkl_Filename, 'wb') as file:  
    pickle.dump(modelE, file)

In [274]:
## Load the Model back from file

with open("Pickle_NBA_XG1_Model2021.pkl" , 'rb') as file:  
    model3 = pickle.load(file)

with open("Pickle_NBA_Linear_Model2021.pkl" , 'rb') as file:  
    regr = pickle.load(file)

## Load the Model back from file
with open("Pickle_NBA_robust_Model2021.pkl" , 'rb') as file:  
    ransac = pickle.load(file)



## Load the Model back from file
with open("Pickle_NBA_lasso_Model2021.pkl" , 'rb') as file:  
    clf = pickle.load(file)



## Load the Model back from file
with open("Pickle_NBA_XG2_Model2021.pkl", 'rb') as file:  
    modelE = pickle.load(file)


In [279]:
x_data["DK points"] =df["DK points"]
x_data["1"] =1

In [280]:
####Build a function to optimze results based on the manual pred method
manualpred = []
for v1, v2, v3,v4 in zip(x_data["DK points"],
        x_data["lasso_Pred"],x_data["xgb_Pred"],x_data["season_DK points"]):
    if v1 <= 7.5:
        manualpred.append(v3)
    elif v1 >= 30.75:
        manualpred.append(v3)
    elif  15 < v1 < 22.25:
        manualpred.append(v4)
    else:
        manualpred.append(v2)
x_data["Man_pred"] =manualpred

In [281]:

def r2_rmse( x_data ):
    r2 = r2_score( x_data['DK points'], x_data['xgb_Pred'] )
    rmse = np.sqrt( mean_squared_error( x_data['DK points'], x_data['xgb_Pred'] ) )
    return pd.Series( dict(  r2 = r2, rmse = rmse ) )

#x_data.groupby(["Dk_bucket"]).apply( r2_rmse ).reset_index()
x_data.groupby("1").apply( r2_rmse ).reset_index()

,1,r2,rmse
0,1,0.562,8.844


In [283]:

def r2_rmse( x_data ):
    r2 = r2_score( x_data['DK points'], x_data['Man_pred'] )
    rmse = np.sqrt( mean_squared_error( x_data['DK points'], x_data['Man_pred'] ) )
    return pd.Series( dict(  r2 = r2, rmse = rmse ) )

#x_data.groupby(["Dk_bucket"]).apply( r2_rmse ).reset_index()
x_data.groupby("1").apply( r2_rmse ).reset_index()

,1,r2,rmse
0,1,0.637,8.050


In [131]:
training["Date"] = df["Date"]
unqdates=training[training["Season"]==2021]["Date"].unique()

In [132]:
datelist =[]
xgpred =[]
manpred =[]

for v1 in unqdates:
    print(v1)
    datelist.append(v1)
    ##################################################################
    x_data = training[[ 'roll2_Age_int','roll7_Age_int' ,'roll2_DK points', 'roll2_Bonus counter',
           'roll2_FTA', 'roll2_FT%', 'roll2_ORB', 'roll2_DRB', 'roll2_TRB',
           'roll2_AST', 'roll2_STL', 'roll2_BLK', 'roll2_TOV', 'roll2_PF',
           'roll2_PTS', 'roll2_GmSc', 'roll2_+/-', 'roll2_TS%', 'roll2_eFG%',
           'roll2_ORB%', 'roll2_DRB%', 'roll2_TRB%', 'roll2_AST%', 'roll2_STL%',
           'roll2_BLK%', 'roll2_TOV%', 'roll2_USG%', 'roll2_ORtg', 'roll2_DRtg',
           'roll2_BPM',  'roll7_DK points', 'roll7_Bonus counter',
           'roll7_FTA', 'roll7_FT%', 'roll7_ORB', 'roll7_DRB', 'roll7_TRB',
           'roll7_AST', 'roll7_STL', 'roll7_BLK', 'roll7_TOV', 'roll7_PF',
           'roll7_PTS', 'roll7_GmSc', 'roll7_+/-', 'roll7_TS%', 'roll7_eFG%',
           'roll7_ORB%', 'roll7_DRB%', 'roll7_TRB%', 'roll7_AST%', 'roll7_STL%',
           'roll7_BLK%', 'roll7_TOV%', 'roll7_USG%', 'roll7_ORtg', 'roll7_DRtg',
           'roll7_BPM',
            'season_Age_int', 'season_DK points', 'season_Bonus counter',
           'season_FTA', 'season_FT%', 'season_ORB', 'season_DRB', 'season_TRB',
           'season_AST', 'season_STL', 'season_BLK', 'season_TOV', 'season_PF',
           'season_PTS', 'season_GmSc', 'season_+/-', 'season_TS%', 'season_eFG%',
           'season_ORB%', 'season_DRB%', 'season_TRB%', 'season_AST%',
           'season_STL%', 'season_BLK%', 'season_TOV%', 'season_USG%',
           'season_ORtg', 'season_DRtg', 'season_BPM'
                      ,"Season","Pos","Wt","Ht","Home/Away","Tm","Opp"]]
    #x_data =x_data[~x_data["Pos"].isin(["SG","SF"])]
    #,"Home/Away","Tm","Opp"
    ##Convert column to dummies
    x_data = pd.get_dummies(x_data, columns=["Season"])
    x_data = pd.get_dummies(x_data, columns=["Pos"])
    x_data = pd.get_dummies(x_data, columns=["Home/Away"])
    x_data = pd.get_dummies(x_data, columns=["Tm"])
    x_data = pd.get_dummies(x_data, columns=["Opp"])
    ###Add in date for filters
    x_data["Date"] = df["Date"]
    x_data =x_data[x_data["Date"]==v1]
    x_data = x_data.drop("Date",axis=1)
    #1st xgx_data
    x_data["xgb_Pred"] = model3.predict(x_data)
    #Linear x_data
    x_data["Linear_Pred"] = regr.predict(x_data)
    #robust x_data
    x_data["robust_Pred"] = ransac.predict(x_data)
    #Lasso x_data
    x_data["lasso_Pred"] = clf.predict(x_data)
    #2nd Xg Final Ensamble
    x_data["xgb_Pred"] = modelE.predict(x_data)
    x_data["DK points"] =df["DK points"]
    x_data["1"] =1
    ####Build a function to optimze results based on the manual pred method
    manualpred = []
    for v1, v2, v3,v4 in zip(x_data["DK points"],
            x_data["lasso_Pred"],x_data["xgb_Pred"],x_data["season_DK points"]):
        if v1 <= 7.5:
            manualpred.append(v3)
        elif v1 >= 30.75:
            manualpred.append(v3)
        elif  15 < v1 < 22.25:
            manualpred.append(v4)
        else:
            manualpred.append(v2)
    x_data["Man_pred"] =manualpred

    def r2_rmse( x_data ):
        r2 = r2_score( x_data['DK points'], x_data['xgb_Pred'] )
        rmse = np.sqrt( mean_squared_error( x_data['DK points'], x_data['xgb_Pred'] ) )
        return pd.Series( dict(  r2 = r2, rmse = rmse ) )

    #x_data.groupby(["Dk_bucket"]).apply( r2_rmse ).reset_index()
    #print(x_data.groupby("1").apply( r2_rmse ).reset_index()["rmse"])
    xgpred.extend(x_data.groupby("1").apply( r2_rmse ).reset_index()["rmse"].to_list())
    def r2_rmse( x_data ):
        r2 = r2_score( x_data['DK points'], x_data['Man_pred'] )
        rmse = np.sqrt( mean_squared_error( x_data['DK points'],
                                           x_data['Man_pred'] ) )
        return pd.Series( dict(  r2 = r2, rmse = rmse ) )

    #x_data.groupby(["Dk_bucket"]).apply( r2_rmse ).reset_index()
    #print(x_data.groupby("1").apply( r2_rmse ).reset_index()["rmse"])
    
    manpred.extend(x_data.groupby("1").apply( r2_rmse ).reset_index()["rmse"].to_list())

2021-01-06T00:00:00.000000000
2020-12-23T00:00:00.000000000
2020-12-25T00:00:00.000000000
2020-12-27T00:00:00.000000000
2020-12-29T00:00:00.000000000
2020-12-31T00:00:00.000000000
2021-01-02T00:00:00.000000000
2021-01-04T00:00:00.000000000
2021-01-08T00:00:00.000000000
2021-01-13T00:00:00.000000000
2021-01-15T00:00:00.000000000
2021-01-17T00:00:00.000000000
2020-12-30T00:00:00.000000000
2021-01-01T00:00:00.000000000
2021-01-09T00:00:00.000000000
2021-01-16T00:00:00.000000000
2020-12-26T00:00:00.000000000
2021-01-05T00:00:00.000000000
2021-01-07T00:00:00.000000000
2021-01-10T00:00:00.000000000
2021-01-12T00:00:00.000000000
2021-01-14T00:00:00.000000000
2021-01-11T00:00:00.000000000
2020-12-28T00:00:00.000000000
2021-01-03T00:00:00.000000000
2020-12-22T00:00:00.000000000


In [133]:
finalresults = pd.DataFrame(list(zip(datelist,xgpred, manpred)), 
               columns =['Date', 'xg_pred',"man_pred"]) 
finalresults.sort_values("Date")

,Date,xg_pred,man_pred
25,2020-12-22,12.975,11.780
1,2020-12-23,11.309,10.661
2,2020-12-25,7.976,7.605
16,2020-12-26,8.856,8.560
3,2020-12-27,9.160,8.464
23,2020-12-28,9.055,8.604
4,2020-12-29,9.696,9.339
12,2020-12-30,9.448,8.715
5,2020-12-31,9.137,8.600
13,2021-01-01,8.505,7.962


In [470]:
####Read in the draftkings data
dkdata = pd.read_csv("master_draftkings_playerdata.csv")
newname =[]

for v1 in dkdata["Name"]:
    newname.append(unidecode.unidecode(v1))
dkdata["clean_player"] =newname
dkdata["clean_player"] =dkdata["clean_player"].str.replace(".","")
dkdata["clean_player"] =dkdata["clean_player"].str.replace("III","")
dkdata["clean_player"] =dkdata["clean_player"].str.replace("II","")
dkdata["clean_player"] =dkdata["clean_player"].str.replace('(TW)','')
dkdata["clean_player"] =dkdata["clean_player"].str.replace("IV","")
dkdata["clean_player"] =dkdata["clean_player"].str.replace("Sr","")
dkdata["clean_player"] =dkdata["clean_player"].str.replace("(",'')
dkdata["clean_player"] =dkdata["clean_player"].str.replace(")",'')
dkdata["clean_player"] =dkdata["clean_player"].str.replace(" Jr",'')
dkdata["clean_player"] =dkdata["clean_player"].str.lower()
dkdata["clean_player"] =dkdata["clean_player"].str.strip()
print(dkdata.shape)
dkdata.head()

(3727, 12)


,Position,Name + ID,Name,ID,Roster Position,Salary,Game Info,TeamAbbrev,AvgPointsPerGame,Year,Date_pulled,clean_player
0,PG,Trae Young (16082060),Trae Young,16082060,PG/G/UTIL,10200,CLE@ATL 01/02/2021 07:30PM ET,ATL,48.850,2021,1/2/2021,trae young
1,C,Joel Embiid (16082063),Joel Embiid,16082063,C/UTIL,10000,CHA@PHI 01/02/2021 07:00PM ET,PHI,50.690,2021,1/2/2021,joel embiid
2,PF/C,Domantas Sabonis (16082065),Domantas Sabonis,16082065,PF/C/F/UTIL,9400,NY@IND 01/02/2021 07:00PM ET,IND,50.550,2021,1/2/2021,domantas sabonis
3,C,Andre Drummond (16082069),Andre Drummond,16082069,C/UTIL,9200,CLE@ATL 01/02/2021 07:30PM ET,CLE,51.300,2021,1/2/2021,andre drummond
4,PF/C,Julius Randle (16082071),Julius Randle,16082071,PF/C/F/UTIL,9100,NY@IND 01/02/2021 07:00PM ET,NY,48.200,2021,1/2/2021,julius randle


In [471]:
##Read in the player list 
pdf = pd.read_csv("NBA_Player_List_2015_2021.csv")
print(pdf.shape)
pdf =pdf[["Player","bbref_code","Team_ABV","Season"]]
####Get the newest datapoint for all
idx = pdf.groupby(['bbref_code'])['Season'].transform(max) == pdf['Season']
pdf[idx]=pdf[idx]
####
newname =[]

for v1 in pdf["Player"]:
    newname.append(unidecode.unidecode(v1))
pdf["clean_player"] =newname
####
pdf.head()

(3966, 12)


,Player,bbref_code,Team_ABV,Season,clean_player
0,Pero Antić,anticpe01,ATL,2015,Pero Antic
1,Kent Bazemore,bazemke01,ATL,2015,Kent Bazemore
2,Elton Brand,brandel01,ATL,2015,Elton Brand
3,DeMarre Carroll,carrode01,ATL,2015,DeMarre Carroll
4,Austin Daye,dayeau01,ATL,2015,Austin Daye


In [472]:
pdf["clean_player"] =pdf["clean_player"].str.replace(".","")
pdf["clean_player"] =pdf["clean_player"].str.replace("III","")
pdf["clean_player"] =pdf["clean_player"].str.replace("II","")
pdf["clean_player"] =pdf["clean_player"].str.replace("IV","")
pdf["clean_player"] =pdf["clean_player"].str.replace("Sr","")
pdf["clean_player"] =pdf["clean_player"].str.replace('(TW)','')
pdf["clean_player"] =pdf["clean_player"].str.replace("(",'')
pdf["clean_player"] =pdf["clean_player"].str.replace(")",'')
pdf["clean_player"] =pdf["clean_player"].str.replace(" Jr",'')
pdf["clean_player"] =pdf["clean_player"].str.lower()
pdf["clean_player"] =pdf["clean_player"].str.strip()

In [478]:
pdf[pdf["bbref_code"]=="hernawi01"]

,Player,bbref_code,Team_ABV,Season,clean_player
405,Willy Hernangómez,hernawi01,CHO,2018,willy hernangomez
424,Willy Hernangómez,hernawi01,CHO,2019,willy hernangomez
441,Willy Hernangómez,hernawi01,CHO,2020,willy hernangomez
2259,Willy Hernangómez,hernawi01,NYK,2017,willy hernangomez
2278,Willy Hernangómez,hernawi01,NYK,2018,willy hernangomez
3778,Willy Hernangómez,hernawi01,NOP,2021,willy hernangomez


In [542]:
addinbbref=dkdata.merge(pdf,left_on=("clean_player"),
                        right_on=("clean_player"),how='left')
addinbbref =addinbbref[addinbbref["Game Info"]!="Postponed"]

In [543]:
####Check how much we are missing
addinbbref[addinbbref["bbref_code"].isnull()].shape

(37, 16)

In [544]:
addinbbref[addinbbref["bbref_code"].isnull()].groupby(["Name","clean_player"]).count()

,,Position,Name + ID,ID,Roster Position,Salary,Game Info,TeamAbbrev,AvgPointsPerGame,Year,Date_pulled,Player,bbref_code,Team_ABV,Season
Name,clean_player,,,,,,,,,,,,,,
Guillermo Hernangomez,guillermo hernangomez,8,8,8,8,8,8,8,8,8,8,0,0,0,0
Juancho Hernangomez,juancho hernangomez,6,6,6,6,6,6,6,6,6,6,0,0,0,0
KJ Martin Jr.,kj martin,6,6,6,6,6,6,6,6,6,6,0,0,0,0
Moe Harkless,moe harkless,5,5,5,5,5,5,5,5,5,5,0,0,0,0
Svi Mykhailiuk,svi mykhailiuk,6,6,6,6,6,6,6,6,6,6,0,0,0,0
Wes Iwundu,wes iwundu,6,6,6,6,6,6,6,6,6,6,0,0,0,0


In [565]:
final_preds = x_data[["Man_pred","xgb_Pred","roll2_DK points","DK points"]]
final_preds["bbref_code"]= df["bbref_code"]
final_preds["Date"]= df["Date"]
final_preds.head()

<ipython-input-565-4fe5a1ebb4f5>:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-565-4fe5a1ebb4f5>:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,Man_pred,xgb_Pred,roll2_DK points,DK points,bbref_code,Date
row_num,,,,,,
311990,19.799,17.461,17.500,17.000,harrijo01,2020-12-27
312003,16.160,15.092,5.750,14.000,luwawti01,2020-12-27
312016,13.126,13.126,8.750,3.000,greenje02,2020-12-27
312029,14.745,14.745,10.500,-0.250,shamela01,2020-12-27
312042,19.798,18.737,20.500,20.000,jordade01,2020-12-27


In [546]:
Extractdate =[]
for v1 in addinbbref["Game Info"]:
    Extractdate.append(datetime.datetime.strptime((v1.split(" ")[1]),'%m/%d/%Y'))
addinbbref["Date"]=Extractdate

In [522]:
#addinbbref["Date"]=pd.to_datetime(addinbbref["Date_pulled"],
#                                format= '%Y-%m-%d')#,errors = 'ignore'
addinbbref["Date"] =addinbbref["Date_pulled"].astype(str)
addinbbref["Date"]=pd.to_datetime(addinbbref["Date_pulled"],
                                format= '%m/%d/%Y',errors = 'ignore')
addinbbref["Date"]=pd.to_datetime(addinbbref["Date_pulled"],
                                format= '%Y%m%d',errors = 'ignore')#

In [547]:
addinbbref.dtypes

Position                    object
Name + ID                   object
Name                        object
ID                           int64
Roster Position             object
Salary                       int64
Game Info                   object
TeamAbbrev                  object
AvgPointsPerGame           float64
Year                         int64
Date_pulled                 object
clean_player                object
Player                      object
bbref_code                  object
Team_ABV                    object
Season                     float64
Date                datetime64[ns]
dtype: object

In [556]:
addinbbref.head(1)#.shape#.head(1)

,Position,Name + ID,Name,ID,Roster Position,Salary,Game Info,TeamAbbrev,AvgPointsPerGame,Year,Date_pulled,clean_player,Player,bbref_code,Team_ABV,Season,Date
0,PG,Trae Young (16082060),Trae Young,16082060,PG/G/UTIL,10200,CLE@ATL 01/02/2021 07:30PM ET,ATL,48.850,2021,1/2/2021,trae young,Trae Young,youngtr01,ATL,2019.000,2021-01-02


In [553]:
final_preds.head(1)

,Man_pred,xgb_Pred,roll2_DK points,DK points,bbref_code,Date
row_num,,,,,,
311990,19.799,17.461,17.500,17.000,harrijo01,2020-12-27


In [563]:
print(final_preds["Date"].max())
print(final_preds["Date"].max())

2020-12-27 00:00:00
2020-12-27 00:00:00


In [564]:
print(addinbbref["Date"].max())
print(addinbbref["Date"].max())

2021-01-17 00:00:00
2021-01-17 00:00:00


In [569]:

builddf =addinbbref.merge(final_preds,left_on=("bbref_code","Date")
                 ,right_on=("bbref_code","Date"),how='left')
builddf =builddf[~builddf["Man_pred"].isnull()]
print(builddf.shape)

(0, 21)


In [560]:
builddf[~builddf["Man_pred"].isnull()].head()

,Position,Name + ID,Name,ID,Roster Position,Salary,Game Info,TeamAbbrev,AvgPointsPerGame,Year,Date_pulled,clean_player,Player,bbref_code,Team_ABV,Season,Date_x,Man_pred,xgb_Pred,roll2_DK points,DK points,Date_y
8,PF/C,Domantas Sabonis (16082065),Domantas Sabonis,16082065,PF/C/F/UTIL,9400,NY@IND 01/02/2021 07:00PM ET,IND,50.550,2021,1/2/2021,domantas sabonis,Domantas Sabonis,sabondo01,IND,2018.000,2021-01-02,42.549,42.549,56.625,41.500,2020-12-27
9,PF/C,Domantas Sabonis (16082065),Domantas Sabonis,16082065,PF/C/F/UTIL,9400,NY@IND 01/02/2021 07:00PM ET,IND,50.550,2021,1/2/2021,domantas sabonis,Domantas Sabonis,sabondo01,IND,2019.000,2021-01-02,42.549,42.549,56.625,41.500,2020-12-27
10,PF/C,Domantas Sabonis (16082065),Domantas Sabonis,16082065,PF/C/F/UTIL,9400,NY@IND 01/02/2021 07:00PM ET,IND,50.550,2021,1/2/2021,domantas sabonis,Domantas Sabonis,sabondo01,IND,2020.000,2021-01-02,42.549,42.549,56.625,41.500,2020-12-27
11,PF/C,Domantas Sabonis (16082065),Domantas Sabonis,16082065,PF/C/F/UTIL,9400,NY@IND 01/02/2021 07:00PM ET,IND,50.550,2021,1/2/2021,domantas sabonis,Domantas Sabonis,sabondo01,OKC,2017.000,2021-01-02,42.549,42.549,56.625,41.500,2020-12-27
12,PF/C,Domantas Sabonis (16082065),Domantas Sabonis,16082065,PF/C/F/UTIL,9400,NY@IND 01/02/2021 07:00PM ET,IND,50.550,2021,1/2/2021,domantas sabonis,Domantas Sabonis,sabondo01,IND,2021.000,2021-01-02,42.549,42.549,56.625,41.500,2020-12-27


In [ ]:
"Date"

In [558]:
addinbbref.groupby(["ID"]).agg({"bbref_code":'nunique'}).sort_values("bbref_code",ascending=0).head()

,bbref_code
ID,
16082060,1
16139450,1
16139206,1
16139210,1
16139213,1


In [285]:
df.head()

,bbref_code,bbref_letter,season,G,Date,Age,Age_int,Tm,Home/Away,Opp,Outcome,GS,DK points,Bonus counter,MP,seconds_played,FG,FGA,FG%,3P,3PA,3P%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,GmSc,+/-,TS%,eFG%,ORB%,DRB%,TRB%,AST%,STL%,BLK%,TOV%,USG%,ORtg,DRtg,BPM,Season,Pos,Wt,Ht
row_num,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,anticpe01,a,2015.000,nan,2014-10-29,32-092,11772,ATL,1,TOR,L (-7),Did Not Play,nan,0.000,NaN,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,2015,C,260.000,83
1,anticpe01,a,2015.000,1.000,2014-11-01,32-095,11775,ATL,0,IND,W (+10),0,11.250,0.000,20:07,1207.000,2.000,4.000,0.500,1.000,3.000,0.333,2.000,3.000,0.667,0.000,1.000,1.000,2.000,0.000,0.000,1.000,3.000,7.000,4.100,-2.000,0.658,0.625,0.000,5.300,2.900,15.800,0.000,0.000,15.800,15.800,114.000,109.000,-2.800,2015,C,260.000,83
2,anticpe01,a,2015.000,2.000,2014-11-05,32-099,11779,ATL,1,SAS,L (-2),0,11.250,0.000,15:40,940.000,3.000,7.000,0.429,1.000,3.000,0.333,0.000,0.000,nan,0.000,3.000,3.000,0.000,0.000,0.000,0.000,2.000,7.000,3.400,3.000,0.500,0.500,0.000,24.200,10.600,0.000,0.000,0.000,0.000,19.500,96.000,104.000,-2.600,2015,C,260.000,83
3,anticpe01,a,2015.000,3.000,2014-11-07,32-101,11781,ATL,1,CHO,L (-3),0,18.000,0.000,22:59,1379.000,3.000,7.000,0.429,2.000,5.000,0.400,0.000,0.000,nan,0.000,2.000,2.000,3.000,1.000,0.000,0.000,3.000,8.000,6.800,16.000,0.571,0.571,0.000,12.000,5.700,21.400,2.200,0.000,0.000,14.300,124.000,107.000,6.100,2015,C,260.000,83
4,anticpe01,a,2015.000,4.000,2014-11-08,32-102,11782,ATL,0,NYK,W (+7),0,5.500,0.000,17:39,1059.000,0.000,3.000,0.000,0.000,2.000,0.000,2.000,3.000,0.667,1.000,1.000,2.000,0.000,0.000,1.000,2.000,2.000,2.000,-1.600,21.000,0.231,0.000,6.300,6.500,6.400,0.000,0.000,4.300,31.600,16.400,46.000,113.000,-17.400,2015,C,260.000,83


In [264]:
x_data[]]

,roll2_Age_int,roll7_Age_int,roll2_DK points,roll2_Bonus counter,roll2_FTA,roll2_FT%,roll2_ORB,roll2_DRB,roll2_TRB,roll2_AST,roll2_STL,roll2_BLK,roll2_TOV,roll2_PF,roll2_PTS,roll2_GmSc,roll2_+/-,roll2_TS%,roll2_eFG%,roll2_ORB%,roll2_DRB%,roll2_TRB%,roll2_AST%,roll2_STL%,roll2_BLK%,roll2_TOV%,roll2_USG%,roll2_ORtg,roll2_DRtg,roll2_BPM,roll7_DK points,roll7_Bonus counter,roll7_FTA,roll7_FT%,roll7_ORB,roll7_DRB,roll7_TRB,roll7_AST,roll7_STL,roll7_BLK,roll7_TOV,roll7_PF,roll7_PTS,roll7_GmSc,roll7_+/-,roll7_TS%,roll7_eFG%,roll7_ORB%,roll7_DRB%,roll7_TRB%,roll7_AST%,roll7_STL%,roll7_BLK%,roll7_TOV%,roll7_USG%,roll7_ORtg,roll7_DRtg,roll7_BPM,season_Age_int,season_DK points,season_Bonus counter,season_FTA,season_FT%,season_ORB,season_DRB,season_TRB,season_AST,season_STL,season_BLK,season_TOV,season_PF,season_PTS,season_GmSc,season_+/-,season_TS%,season_eFG%,season_ORB%,season_DRB%,season_TRB%,season_AST%,season_STL%,season_BLK%,season_TOV%,season_USG%,season_ORtg,season_DRtg,season_BPM,Wt,Ht,Season_2015,Season_2016,Season_2017,Season_2018,Season_2019,Season_2020,Season_2021,Pos_C,Pos_C-F,Pos_F,Pos_G,Pos_G-F,Pos_PF,Pos_PG,Pos_SF,Pos_SG,Home/Away_0,Home/Away_1,Tm_ATL,Tm_BOS,Tm_BRK,Tm_CHI,Tm_CHO,Tm_CLE,Tm_DAL,Tm_DEN,Tm_DET,Tm_GSW,Tm_HOU,Tm_IND,Tm_LAC,Tm_LAL,Tm_MEM,Tm_MIA,Tm_MIL,Tm_MIN,Tm_NOP,Tm_NYK,Tm_OKC,Tm_ORL,Tm_PHI,Tm_PHO,Tm_POR,Tm_SAC,Tm_SAS,Tm_TOR,Tm_UTA,Tm_WAS,Opp_ATL,Opp_BOS,Opp_BRK,Opp_CHI,Opp_CHO,Opp_CLE,Opp_DAL,Opp_DEN,Opp_DET,Opp_GSW,Opp_HOU,Opp_IND,Opp_LAC,Opp_LAL,Opp_MEM,Opp_MIA,Opp_MIL,Opp_MIN,Opp_NOP,Opp_NYK,Opp_OKC,Opp_ORL,Opp_PHI,Opp_PHO,Opp_POR,Opp_SAC,Opp_SAS,Opp_TOR,Opp_UTA,Opp_WAS,xgb_Pred,Linear_Pred,robust_Pred,lasso_Pred,DK points,1,Man_pred
row_num,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
312330,9791.000,9786.571,26.250,0.000,1.500,0.667,0.000,6.000,6.000,1.000,1.000,0.000,1.500,2.500,15.000,9.100,4.000,0.572,0.562,0.000,16.400,8.650,4.350,1.450,0.000,11.500,18.850,99.000,103.000,-4.050,32.214,0.000,3.286,0.761,0.286,3.714,4.000,2.429,1.714,0.429,1.571,2.000,18.429,14.114,0.000,0.623,0.589,0.943,11.186,6.171,12.329,2.543,1.314,9.986,21.200,114.429,109.857,1.971,9829.427,19.802,0.079,2.031,0.753,0.920,3.031,3.951,2.076,0.703,0.425,1.239,1.869,9.537,7.306,-0.339,0.526,0.494,4.655,14.904,9.798,13.342,1.527,1.604,12.929,18.945,102.202,109.673,-1.609,190.000,73,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,26.023,26.593,-1.905,24.151,39.750,1,26.023
312343,8172.000,8167.571,32.875,0.750,3.000,0.875,2.500,5.000,7.500,4.000,0.500,0.500,1.500,3.000,14.500,10.700,1.000,0.478,0.434,8.250,15.000,11.500,20.250,0.750,1.550,8.700,23.600,103.500,104.500,-0.800,32.679,0.214,3.429,0.893,1.714,5.286,7.000,3.571,1.000,1.714,1.429,3.000,12.857,11.900,-0.714,0.535,0.469,6.529,15.871,11.614,18.786,1.743,5.243,10.929,19.500,113.000,108.143,1.243,9829.363,19.803,0.079,2.031,0.753,0.920,3.031,3.951,2.076,0.703,0.425,1.239,1.869,9.537,7.306,-0.339,0.526,0.494,4.655,14.904,9.798,13.342,1.527,1.604,12.929,18.945,102.202,109.673,-1.609,230.000,79,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,31.895,32.210,21.661,27.728,41.750,1,31.895
312356,9450.000,9445.571,33.750,0.000,2.000,0.750,0.000,1.000,1.000,7.000,3.000,0.000,1.000,1.500,15.500,14.150,-2.000,0.498,0.481,0.000,2.800,1.600,31.700,4.300,0.000,6.150,20.900,109.500,103.000,2.550,31.071,0.000,3.571,0.867,0.429,2.714,3.143,6.571,1.857,0.000,1.143,1.571,13.000,11.514,-0.286,0.462,0.386,1.329,8.557,5.071,28.414,2.600,0.000,7.529,19.886,107.571,112.000,-1.371,9829.348,19.803,0.079,2.031,0.753,0.920,3.031,3.951,2.077,0.703,0.425,1.239,1.869,9.537,7.306,-0.339,0.526,0.494,4.655,14.903,9.798,13.343,1.527,1.604,12.928,18.945,102.202,109.673,-1.609,